## Домашняя работа №2
### С помощью кода, аналогичного приведенному в материалах, посмотрите на SNP, появляющиеся в вашем гене из 1 задания. Загрузите 3 любых понравившихся варианта, как описано.
### В таблицу гена, полученную по итогам практического задания первого урока, добавьте следующую информацию: количество rs для выбранного гена.

In [65]:
import pandas as pd

In [24]:
from Bio.Seq import MutableSeq
seq_sl = 'CTCTCGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCGGAGAGCGAGGGGAGGCGGAGACGGAGGAAGGTCTGAGGAGCAGCTTCAGTCCCC'
mutable_seq = MutableSeq(seq_sl)

In [25]:
mutable_seq.transcribe().translate()

MutableSeq('LSVCGSSVGPAPGAESEGRRRRRKV*GAASVP')

In [26]:
mutable_seq[4] = "A"

In [27]:
mutable_seq

MutableSeq('CTCTAGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCGGAGAGCGAGGGG...CCC')

In [30]:
mutable_seq.transcribe().translate()

MutableSeq('L*VCGSSVGPAPGAESEGRRRRRKV*GAASVP')

In [31]:
# install biopython on Jupyter server.
import sys
#!python -m pip install biopython

In [13]:
import time
from Bio import Entrez

In [55]:
Entrez.email = "aspirantsv1@gmail.com" # provide your user email 
# RECOMMENDED: apply for API key from NCBI (https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/). 
# 10 queries per second with a valid API key, otherwise 3 queries per seconds are allowed for 'None'
Entrez.api_key = None

# dbSNP supported query terms (https://www.ncbi.nlm.nih.gov/snp/docs/entrez_help/) can be build and test online using web query builder (https://www.ncbi.nlm.nih.gov/snp/advanced) 
# esearch handle
eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          #complex query for missense and pathogenic variants in LPL gene with global MAF betweeen 0 and 0.01.
                          term='CTNNB1[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class]', 
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )


In [56]:

# get esearch result
eSresult = Entrez.read(eShandle)

In [57]:
# review results 
for k in eSresult:
    print (k, ":", eSresult[k])
    
#Output: Web environment (&WebEnv) and query key (&query_key) parameters specifying the location on the Entrez history server of the list of UIDs matching the Entrez query
#https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Storing_Search_Results
    

Count : 59
RetMax : 20
RetStart : 0
QueryKey : 1
WebEnv : MCID_63e12ef030dc2358b63c0740
IdList : ['2125628072', '2125623075', '2125620433', '2125617982', '2125617859', '2078436466', '2078162308', '2078159952', '2078151590', '2078133150', '1575334103', '1559474557', '1559474364', '1559474040', '1553632412', '1553631860', '1553630472', '1553630279', '1454068577', '1064796453']
TranslationSet : []
TranslationStack : [{'Term': 'CTNNB1[All Fields]', 'Field': 'All Fields', 'Count': '19470', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '130269', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '10256260', 'Explode': 'N'}, 'AND']
QueryTranslation : CTNNB1[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class]


In [58]:
# get result RSIDs list 'Idlist'
# total rs count 
rslist = (eSresult['IdList'])
rslist

['2125628072', '2125623075', '2125620433', '2125617982', '2125617859', '2078436466', '2078162308', '2078159952', '2078151590', '2078133150', '1575334103', '1559474557', '1559474364', '1559474040', '1553632412', '1553631860', '1553630472', '1553630279', '1454068577', '1064796453']

In [59]:
# retmax = 20 so print only 20 RSIDs
# additional results can be retrieved by batches
# download in batches example http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139 or see below.
for rs in rslist:
    print(rs)

2125628072
2125623075
2125620433
2125617982
2125617859
2078436466
2078162308
2078159952
2078151590
2078133150
1575334103
1559474557
1559474364
1559474040
1553632412
1553631860
1553630472
1553630279
1454068577
1064796453


In [60]:
# get the WebEnv session cookie, and the QueryKey:

webenv = eSresult["WebEnv"]
query_key = eSresult["QueryKey"]
total_count = int(eSresult["Count"])
query_key = eSresult["QueryKey"]
retmax = 2 # return 2 rs per batch example

Чтобы не получить большого вывода информации поставлю вместо total_count, 10

In [64]:
# sample codes adopted with modifications from http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139.
fetch_count = 0
for start in range(0, 10, retmax):
    end = min(total_count, start+retmax)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    #fetch_count += 1
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="snp",
                                         #rettype="uilist", #available types [uilist | xml (use retmode=xml))
                                         retmode="xml",
                                         retstart=start,
                                         retmax=retmax,
                                         webenv=webenv,
                                         query_key=query_key )
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    if (fetch_handle):
        #print(fetch_handle)            
        data = fetch_handle.read()
        print(data)
        fetch_handle.close()


Going to download record 1 to 2
b'<?xml version="1.0" ?>\n<ExchangeSet xmlns:xsi="https://www.w3.org/2001/XMLSchema-instance" xmlns="https://www.ncbi.nlm.nih.gov/SNP/docsum" xsi:schemaLocation="https://www.ncbi.nlm.nih.gov/SNP/docsum ftp://ftp.ncbi.nlm.nih.gov/snp/specs/docsum_eutils.xsd" ><DocumentSummary uid="2125628072"><SNP_ID>2125628072</SNP_ID><ALLELE_ORIGIN/><GLOBAL_MAFS/><GLOBAL_POPULATION/><GLOBAL_SAMPLESIZE>0</GLOBAL_SAMPLESIZE><SUSPECTED/><CLINICAL_SIGNIFICANCE>pathogenic</CLINICAL_SIGNIFICANCE><GENES><GENE_E><NAME>CTNNB1</NAME><GENE_ID>1499</GENE_ID></GENE_E></GENES><ACC>NC_000003.12</ACC><CHR>3</CHR><HANDLE>EVA</HANDLE><SPDI>NC_000003.12:41227303:A:G,NC_000003.12:41227303:A:T</SPDI><FXN_CLASS>missense_variant,coding_sequence_variant,stop_gained</FXN_CLASS><VALIDATED/><DOCSUM>HGVS=NC_000003.12:g.41227304A&gt;G,NC_000003.12:g.41227304A&gt;T,NC_000003.11:g.41268795A&gt;G,NC_000003.11:g.41268795A&gt;T,NG_013302.2:g.32854A&gt;G,NG_013302.2:g.32854A&gt;T,NM_001904.4:c.1033A&gt;G

In [74]:
# Выведем итоговую таблицу
pd.read_csv('final.csv').append({'Unnamed: 0': 'rs', 'CTNNB1': int(eSresult['Count'])}, ignore_index = True)

C:\Users\60053972\AppData\Local\Temp\ipykernel_4248\4157096100.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd.read_csv('final.csv').append({'Unnamed: 0': 'rs', 'CTNNB1': int(eSresult['Count'])}, ignore_index = True)


,Unnamed: 0,CTNNB1
0,id,1499
1,accession,NC_000003
2,version,12
3,interval_from,41199505
4,interval_to,41240443
5,prot.seq.example,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...
6,gr_acid_num,781
7,num_of_transcripts,14
8,rs,59
